# Capacity expansion of power generation
[![capacity_expansion.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/miscellaneous/capacity_expansion.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/miscellaneous/capacity_expansion.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/miscellaneous/capacity_expansion.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/miscellaneous/capacity_expansion.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/miscellaneous/capacity_expansion.ipynb)

Description: Models the extensive form of a deterministic multi-stage capacity expansion problem. In this model we can have multiple resources of the same type which have identical properties. The model can be further developed into a stochastic one.

Tags: ampl-only, planning, mip, power-generation

Notebook author: Gyorgy Matyasfalvi

Model author: Ahmed et al., Gyorgy Matyasfalvi

References:
1. Ahmed, S., King, A. J. and Parija G. (2003). "A Multi-Stage Stochastich Integer Programming Approach for Capacity Expansion Under Uncertainty". In: J. of Global Optimization 26.1 (May 2003), pp. 3-24. URL: https://doi.org/10.1023/A:1023062915106

In [1]:
# Install dependencies
!pip install -q amplpy ampltools

In [2]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

*******************************************************************************
* Please make sure that the AMPL folder is in the system search path, or      *
* specify the path via:                                                       *
*     AMPL(Environment('full path to the AMPL installation directory'))       *
*******************************************************************************


RuntimeError: AMPL could not be started. Message from process thread:
cannot execute ampl: No such file or directory



### Use `%%ampl_eval` to evaluate AMPL commands

In [3]:
%%ampl_eval
option version;

UsageError: Cell magic `%%ampl_eval` not found.


### Use %%writeifile to create files

In [4]:
%%writefile cap_ef.mod

# Declare index sets
set STAGES ordered;
set RESOURCES ordered;

# Decision variables
var cap_exp {STAGES, RESOURCES} >= 0;        # Variable cost associated with increase in capacity
var cap_dec {STAGES, RESOURCES} integer; # Fixed cost associated with decision to involve a new resource

# Parameters
param var_cost {STAGES, RESOURCES} > 0;
param fix_cost {STAGES, RESOURCES} > 0;
param cap_ub {STAGES, RESOURCES} > 0;
param demand {STAGES};

# Objective function
minimize total_cost: 
    sum{t in STAGES, i in RESOURCES} (var_cost[t,i]*cap_exp[t,i] + fix_cost[t,i]*cap_dec[t,i]);

# Constraints
subject to cap_acqu_bound {t in STAGES, i in RESOURCES}:
        cap_exp[t,i] <= cap_ub[t,i]*cap_dec[t,i]; # Upper bounds on total capacity

subject to statisfy_demand {T in STAGES}:
        sum {t in STAGES, i in RESOURCES: ord(t) <= ord(T)} cap_exp[t,i] >= demand[T]; # Demand needs to be satisfied at all times

subject to one_resource:
        sum {t in STAGES} cap_dec[t,first(RESOURCES)] <= 1;


Writing cap_ef.mod


In [5]:
%%writefile cap_ef.dat

# Index sets
set STAGES := 2030 2040 2050;
set RESOURCES := hydro nuclear coal;

# Billions of USD
param var_cost:    hydro  nuclear  coal   :=
            2030   0.01   0.01    0.8
            2040   0.011  0.013   1.9 
            2050   0.012  0.014   2.7    ;

# Billions of USD
param fix_cost:    hydro  nuclear  coal   :=
            2030   25     10       1
            2040   28     11       2.3 
            2050   25     11.5     3.8  ;

# In GWH 
param cap_ub:      hydro  nuclear  coal   :=
            2030   300    12       20
            2040   290    15       21 
            2050   280    18       20   ;

# In GWH
param demand := 
            2030 500 
            2040 1000 
            2050 1105   ;

Writing cap_ef.dat


In [6]:
%%writefile cap_ef.run
# ---------------------------------------------------------------
# Solve capcity expansion problem then display decison variables
# ---------------------------------------------------------------

option solver cbc;
option solution_round 6;

model cap_ef.mod;
data cap_ef.dat;
solve;

for {t in STAGES} {
    printf "\n\n\t\t**%s**\n", t;
    printf {i in RESOURCES}: "%s: %d\t", i, cap_dec[t,i];
    printf "\n";
}
printf "\n";

Writing cap_ef.run


### Use `%%ampl_eval` to run the script cut2.run

In [7]:
%%ampl_eval
commands cap_ef.run;

UsageError: Cell magic `%%ampl_eval` not found.
